<a href="https://colab.research.google.com/github/YahyaAlaaMassoud/cv-iti-24/blob/main/lenet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install torchsummary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


![](https://miro.medium.com/v2/resize:fit:2000/1*1TI1aGBZ4dybR6__DI9dzA.png)

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


# Define the transformation of the data
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resizing the image from 28x28 to 32x32 to fit LeNet5 input
    transforms.ToTensor(),        # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the data
])

# Download and load the training data
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)

# Download and load the test data
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15993543.16it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 487385.21it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4382538.64it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4003894.23it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        # Fully connected layers
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)
        # Average Pooling layer
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        # Applying the first convolution and pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        # Applying the second convolution and pooling layers
        x = self.pool(F.relu(self.conv2(x)))
        # Flattening the output for the fully connected layer
        x = x.view(-1, 16 * 5 * 5)
        # Applying the fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Create the LeNet5 model instance
model = LeNet5()

# Summary of LeNet5 using torchsummary
summary(model, input_size=(1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         AvgPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         AvgPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
----------------------------------------------------------------


In [ ]:
# Define the loss function (Cross-Entropy Loss)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Define the optimizer (Adam)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def evaluate_model(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient computation
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    return accuracy


def train_model(model, device, train_loader, test_loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if (batch_idx + 1) % 100 == 0:
                print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {loss.item()}")

        # Evaluate after each epoch
        accuracy = evaluate_model(model, device, test_loader)
        print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")

# Check for device availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Start the training process
train_model(model, device, train_loader, test_loader, optimizer, criterion, epochs=2)

Epoch 1, Batch 100, Loss: 0.4580603241920471
Epoch 1, Batch 200, Loss: 0.2825716733932495
Epoch 1, Batch 300, Loss: 0.5390952229499817
Epoch 1, Batch 400, Loss: 0.05402418598532677
Epoch 1, Batch 500, Loss: 0.26370832324028015
Epoch 1, Batch 600, Loss: 0.048294294625520706
Epoch 1, Batch 700, Loss: 0.01751197688281536
Epoch 1, Batch 800, Loss: 0.0810914859175682
Epoch 1, Batch 900, Loss: 0.06326361000537872
Epoch 1, Loss: 0.0946, Accuracy: 97.15%
Epoch 2, Batch 100, Loss: 0.04591609910130501
Epoch 2, Batch 200, Loss: 0.06341537088155746
Epoch 2, Batch 300, Loss: 0.05889482796192169
Epoch 2, Batch 400, Loss: 0.07080835849046707
Epoch 2, Batch 500, Loss: 0.06106925755739212
Epoch 2, Batch 600, Loss: 0.02593400329351425
Epoch 2, Batch 700, Loss: 0.012323077768087387
Epoch 2, Batch 800, Loss: 0.032192230224609375
Epoch 2, Batch 900, Loss: 0.03057936206459999
Epoch 2, Loss: 0.0397, Accuracy: 98.16%
